# Морфологический анализ в Python

В первом полугодии вы учились делать морфологический анализ текстов, запуская **Mystem** с помощью консоли. Это бывает неудобно, поскольку всю остальную обработку текстов зачастую мы делаем в питоне. Однако есть и библиотеки под Python, который позволяют осуществлять морфологический анализ, так сказать, *"не выходя из IDLE"*. К таким библиотекам относятся **pymorphy2** и **pymystem3**.

Как поставить:

``pip install pymystem3``

``pip install pymorphy2``

И проверяем, работает ли:

In [1]:
from pymystem3 import Mystem
import pymorphy2

## pymystem3

**pymystem3** - это просто питоновская обертка для уже известного вам Mystem. Если у вас на компьюетере нет Mystem, библиотека его скачает с сервера Яндекса, а потом будет за вас запускать, а ответ возвращать интерпретатору Python. Что это значит? Что сама библиотека работает так же медленно, как и оригинальная бинарная программа (и даже ещё чуть-чуть медленнее).

[Документация](http://pythonhosted.org/pymystem3/)

Но довольно слов, займёмся делом. Сначала нам надо создать экзепмляр класса анализатора:

In [2]:
m = Mystem()

У этого класса есть два метода:

* lemmatize, возвращающий список лемм,
* и analyze, возвращающий полные разборы в виде словаря.

Возьмем небольшой текст и опробуем на нем эти два метода:

In [13]:
text = 'Живые выступления Делии де Франс - это камерные шоу,' +\
'в которых классические инструменты - фортепиано и арфа - соединяются с электроникой. ' +\
'Темой музыкального исследования Делии становятся пропорции - ' +\
'как много эксперимента нужно поп-музыке и как много попа нужно экспериментальной музыке.'

In [14]:
text

'Живые выступления Делии де Франс - это камерные шоу,в которых классические инструменты - фортепиано и арфа - соединяются с электроникой. Темой музыкального исследования Делии становятся пропорции - как много эксперимента нужно поп-музыке и как много попа нужно экспериментальной музыке.'

In [15]:
lemmas = m.lemmatize(text)
lemmas[:5]

['живой', ' ', 'выступление', ' ', 'делий']

Можно легко собрать текст обратно:

In [16]:
print(''.join(lemmas))

живой выступление делий де франс - это камерный шоу,в который классический инструмент - фортепиано и арфа - соединяться с электроника. тема музыкальный исследование делий становиться пропорция - как много эксперимент нужно поп-музыка и как много поп нужный экспериментальный музыка.



In [17]:
ana = m.analyze(text)
print(ana[:10])

[{'analysis': [{'gr': 'A=(вин,мн,полн,неод|им,мн,полн)', 'lex': 'живой'}], 'text': 'Живые'}, {'text': ' '}, {'analysis': [{'gr': 'S,сред,неод=(вин,мн|род,ед|им,мн)', 'lex': 'выступление'}], 'text': 'выступления'}, {'text': ' '}, {'analysis': [{'gr': 'S,имя,муж,од=(пр,ед|им,мн)', 'lex': 'делий'}], 'text': 'Делии'}, {'text': ' '}, {'analysis': [{'gr': 'PART=', 'lex': 'де'}], 'text': 'де'}, {'text': ' '}, {'analysis': [{'gr': 'S,имя,муж,од=им,ед', 'lex': 'франс'}], 'text': 'Франс'}, {'text': ' - '}]


Разбор для каждого слова является элементом массива:

In [18]:
for word in ana[:10]:
    print(word)

{'analysis': [{'gr': 'A=(вин,мн,полн,неод|им,мн,полн)', 'lex': 'живой'}], 'text': 'Живые'}
{'text': ' '}
{'analysis': [{'gr': 'S,сред,неод=(вин,мн|род,ед|им,мн)', 'lex': 'выступление'}], 'text': 'выступления'}
{'text': ' '}
{'analysis': [{'gr': 'S,имя,муж,од=(пр,ед|им,мн)', 'lex': 'делий'}], 'text': 'Делии'}
{'text': ' '}
{'analysis': [{'gr': 'PART=', 'lex': 'де'}], 'text': 'де'}
{'text': ' '}
{'analysis': [{'gr': 'S,имя,муж,од=им,ед', 'lex': 'франс'}], 'text': 'Франс'}
{'text': ' - '}


В этом разборе в поле `text` можно найти исходное слова, а в поле `analysis` (которого может и не быть) - грамматические характеристики и леммы.

В грамматическом разборе знаком `=` отделяются изменяемые характеристики от неизменяемых. Знаком `|` отделяются омонимичные разборы.

Достанем все части речи:

In [19]:
for word in ana:
    if 'analysis' in word:
        gr = word['analysis'][0]['gr']
        pos = gr.split('=')[0].split(',')[0]
        print(word['text'], pos)

Живые A
выступления S
Делии S
де PART
Франс S
это PART
камерные A
шоу S
в PR
которых APRO
классические A
инструменты S
фортепиано S
и CONJ
арфа S
соединяются V
с PR
электроникой S
Темой S
музыкального A
исследования S
Делии S
становятся V
пропорции S
как CONJ
много ADV
эксперимента S
нужно ADV
поп-музыке S
и CONJ
как ADVPRO
много ADV
попа S
нужно A
экспериментальной A
музыке S


### Подведем итог:

**Достоинства**:

* хорошее качество разбора
* по умолчанию разрешается частеречная омонимия (внутри части речи остается)
* при разборе учитывается контекст
* совместим с разметкой НКРЯ

**Недостатки**:

* медленный
* analyze возвращает неудобный json

## pymorphy2

pymorphy2 - это не обертка, а полноценный морфологический анализатор, целиком написанный на Python. Он может делать то же, что и pymystem3, и даже больше: изменять слова в нужную форму (спрягать и склонять). При этом pymorphy2 справляется и с незнакомыми словами.

[документация](https://pymorphy2.readthedocs.io/en/latest/)

pymorphy2 обучался на словарях проекта OpenCorpora, что накладывается определённый отпечаток на его тагсет (например, он не очень хорошо совместим с тагсетом Mystem).

Для работы точно так же надо создать экземпляр класса `MorphAnalyzer`. Рекомендуется создать один экземпляр и дальше с ним и работать, поскольку он занимает достаточно много памяти, и если создать несколько экземпляров анализаторов, то они будут тормозить программу.

In [20]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

Разбор слова делается при помощи метода parse:

In [21]:
ana = morph.parse('стали')
ana

[Parse(word='стали', tag=OpencorporaTag('VERB,perf,intr plur,past,indc'), normal_form='стать', score=0.984662, methods_stack=((<DictionaryAnalyzer>, 'стали', 904, 4),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,gent'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 1),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,datv'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 2),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,loct'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 5),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,nomn'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 6),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,accs'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 9),))]

Как видно, анализатор вернул все возможные разборы этого слова, отранжировав их по вероятности.

У каждого разбора есть атрибуты: исходное слово, тэг, лемма, вероятность разбора:

In [22]:
first = ana[0]  # первый разбор
print('Слово:', first.word)
print('Тэг:', first.tag)
print('Лемма:', first.normal_form)
print('Вероятность:', first.score)

Слово: стали
Тэг: VERB,perf,intr plur,past,indc
Лемма: стать
Вероятность: 0.984662


Для каждого разбора можно получить лемму и всю информацию о ней (т.е. еще один разбор, только уже для леммы):

In [23]:
first.normalized

Parse(word='стать', tag=OpencorporaTag('INFN,perf,intr'), normal_form='стать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'стать', 904, 0),))

In [24]:
last = ana[-1] # последний разбор
print('Разбор слова: ', last)
print()
print('Разбор леммы: ', last.normalized)

Разбор слова:  Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,accs'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 9),))

Разбор леммы:  Parse(word='сталь', tag=OpencorporaTag('NOUN,inan,femn sing,nomn'), normal_form='сталь', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'сталь', 13, 0),))


Если распечатать тег разбора, то может показаться, что это строка:

In [26]:
first = ana[0]  # первый разбор
print(first.tag)

VERB,perf,intr plur,past,indc


Но на самом деле это объект класса OpencorporaTag, так что некоторые вещи, которые можно делать со строками, с тэгами делать нельзя. А некоторые все-таки можно.

Например, можно проверить, есть ли какая-то граммема в теге:

In [27]:
'NOUN' in first.tag

False

In [28]:
'VERB' in first.tag

True

In [29]:
{'VERB', 'sing'} in first.tag

False

In [30]:
{'VERB', 'plur'} in first.tag

True

Из каждого тега можно достать более дробную информацию. Если граммема есть в разборе, то вернется ее значение, если ее нет, то вернется None.

In [ ]:
p.tag.POS           # Part of Speech, часть речи
p.tag.animacy       # одушевленность
p.tag.aspect        # вид: совершенный или несовершенный
p.tag.case          # падеж
p.tag.gender        # род (мужской, женский, средний)
p.tag.involvement   # включенность говорящего в действие
p.tag.mood          # наклонение (повелительное, изъявительное)
p.tag.number        # число (единственное, множественное)
p.tag.person        # лицо (1, 2, 3)
p.tag.tense         # время (настоящее, прошедшее, будущее)
p.tag.transitivity  # переходность (переходный, непереходный)
p.tag.voice         # залог (действительный, страдательный)

In [33]:
print(first.tag)
print('Время: ', first.tag.tense)
print('Падеж: ', first.tag.case)

VERB,perf,intr plur,past,indc
Время:  past
Падеж:  None


Список граммем, которые используются в модуле, находится здесь - https://pymorphy2.readthedocs.io/en/latest/user/grammemes.html. Если искать какую-то граммему, которой нет в этом списке, возникнет ошибка.

Можно получить _строку_ с кириллическими обозначениями граммем:

In [34]:
first.tag.cyr_repr

'ГЛ,сов,неперех мн,прош,изъяв'

## Словоизменение

Если у нас есть разбор слова, то мы можем это слово поставить в другую форму с помощью функции inflect. Эта функция получает на вход множество граммем и пытается применить их к нашему разбору.

In [35]:
morph.parse('программирую')

[Parse(word='программирую', tag=OpencorporaTag('VERB,impf,tran sing,1per,pres,indc'), normal_form='программировать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'программирую', 168, 1),))]

In [36]:
prog = morph.parse('программирую')[0]
prog.inflect({'plur'})

Parse(word='программируем', tag=OpencorporaTag('VERB,impf,tran plur,1per,pres,indc'), normal_form='программировать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'программируем', 168, 2),))

In [37]:
prog.inflect({'plur', 'past'})

Parse(word='программировали', tag=OpencorporaTag('VERB,impf,tran plur,past,indc'), normal_form='программировать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'программировали', 168, 10),))

In [38]:
prog.inflect({'past'})

Parse(word='программировал', tag=OpencorporaTag('VERB,impf,tran masc,sing,past,indc'), normal_form='программировать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'программировал', 168, 7),))

In [39]:
prog.inflect({'past', 'femn'})

Parse(word='программировала', tag=OpencorporaTag('VERB,impf,tran femn,sing,past,indc'), normal_form='программировать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'программировала', 168, 8),))

### Формы слова

С помощью атрибута lexeme можно получить массив всех форм слова:

In [40]:
prog.lexeme

[Parse(word='программировать', tag=OpencorporaTag('INFN,impf,tran'), normal_form='программировать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'программировать', 168, 0),)),
 Parse(word='программирую', tag=OpencorporaTag('VERB,impf,tran sing,1per,pres,indc'), normal_form='программировать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'программирую', 168, 1),)),
 Parse(word='программируем', tag=OpencorporaTag('VERB,impf,tran plur,1per,pres,indc'), normal_form='программировать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'программируем', 168, 2),)),
 Parse(word='программируешь', tag=OpencorporaTag('VERB,impf,tran sing,2per,pres,indc'), normal_form='программировать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'программируешь', 168, 3),)),
 Parse(word='программируете', tag=OpencorporaTag('VERB,impf,tran plur,2per,pres,indc'), normal_form='программировать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'программируете', 168, 4),)),
 Parse(word='программирует', tag=O

### Согласование слов с числительными

Из документации:
> Слово нужно ставить в разные формы в зависимости от числительного, к которому оно относится. Например: “1 бутявка”, “2 бутявки”, “5 бутявок” Для этих целей используйте метод Parse.make_agree_with_number():

In [41]:
butyavka = morph.parse('бутявка')[0]

In [43]:
butyavka.make_agree_with_number(1).word

'бутявка'

In [44]:
butyavka.make_agree_with_number(2).word

'бутявки'

In [45]:
butyavka.make_agree_with_number(5).word

'бутявок'

### Подведем итог:

**Достоинства**:

* умеет составлять разборы, находить лемму, склонять и спрягать
* генерирует гипотезы для незнакомых слов
* написан полностью на питоне и быстрее, чем Mystem (и есть ускоренная версия с вставками на c++)
* может работать с украинским языком (но словари нужно отдельно устанавливать)

**Недостатки**:

* качество хуже, чем у Mystem
* работает только на уровне отдельных слов (и естественно, не учитывает контекст)